In [1]:
import numpy as np
import open3d as o3d

In [2]:
mesh_path = '../tmp/mesh-/29XXX00010001.ply'

points = o3d.io.read_point_cloud(mesh_path)
# mesh = o3d.io.read_triangle_mesh(mesh_path)
# メッシュデータの表示
o3d.visualization.draw_geometries([points])

In [3]:
def cluster_pcd(pcd):
    cluster = pcd.cluster_dbscan(1, 2)
    point_num = len(cluster)
    clusters = np.asarray(cluster)
    cluster_indices = set(list(clusters))
    filtered_pcds = [[] for _ in cluster_indices]
    for k in cluster_indices:
        filtered_indices = filter(lambda x: clusters[x] == k, range(point_num))
        filtered_pcds[k] = pcd.select_down_sample(list(filtered_indices))
    return filtered_pcds

In [4]:
filtered_pcd = cluster_pcd(points)[0]

filtered_pcd.estimate_normals(
    o3d.geometry.KDTreeSearchParamHybrid(
        radius=0.2,
        max_nn=30))

pcd_center =filtered_pcd.get_center().tolist()
camera_location = pcd_center[:-1]+[pcd_center[-1]*100]
# target_pcd, _ = target_pcd.remove_statistical_outlier(5, 1.5)
# target_pcd.orient_normals_to_align_with_direction()
filtered_pcd.orient_normals_towards_camera_location(camera_location)
filtered_pcd = filtered_pcd.normalize_normals()

In [5]:
# _, seg_plane = filtered_pcd.segment_plane(0.2, 3, 1000)
# o3d.visualization.draw_geometries([filtered_pcd.select_down_sample(seg_plane)])

In [6]:
o3d.visualization.draw_geometries([filtered_pcd])

In [7]:
# mesh, filtered_pcd = filtered_pcd.hidden_point_removal(camera_location, radius=0.2)

In [ ]:
radius = 0.1
radii = [radius*2**i for i in range(7)]
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    filtered_pcd, o3d.utility.DoubleVector(radii))

In [ ]:
o3d.visualization.draw_geometries([mesh])

In [ ]:
# tetras = o3d.geometry.TetraMesh.create_from_point_cloud(points)
# o3d.visualization.draw_geometries([tetras])

In [ ]:
# values = compute_sined_distance(tetras.vertices)
# surface = tetras.extract_triangle_mesh(o3d.utility.DoubleVector(values), level=0.0)
# surface.cumpute_vertex_normals()

In [ ]:
# o3d.visualization.draw_geometries([surface])